In [4]:
import requests
from urllib.request import Request, urlopen
import time
import pandas as pd

In [1]:
api_key = "cc796bcd-0c8c-4a92-94d0-34226c53ec5e"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}

In [3]:
link = f"https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=RGAPI-{api_key}"

response = requests.get(link, headers = headers)
response.status_code


200

In [4]:
summoners_list = response.json()['entries']
summoners_id = [summoners_list[i]['summonerId'] for i in range(len(summoners_list))]

In [5]:
puuids = []

i = 0
while i < len(summoners_id):
    summoner_id = summoners_id[i]
    link_puuid = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key=RGAPI-{api_key}"
    response_puuid = requests.get(link_puuid, headers = headers)
    if response_puuid.status_code == 200:
        puuids.append(response_puuid.json()['puuid'])
    else:
        print(response_puuid.json())
        time.sleep(120)
        continue
    i += 1

KeyboardInterrupt: 

In [8]:
temp = pd.DataFrame({"puuids": puuids})

In [9]:
temp.to_csv("challengers_puuids.csv", index = False)

In [3]:
df = pd.read_csv("challengers_puuids.csv")

In [7]:
df

,puuids
0,sMldn-KsHSWZ0c_HYlCIDgai-6IELScbuvszb5Xg1nWIm7...
1,PjqaHqL-1JO3mmGoCdvs2XIdRElh29m0eTgDFuNtHNC-Gx...
2,VnItYUCBIrK1b_n_gfHy4QAxAJhLYa8o5pMUEnUa3F5UdV...
3,S3rHEPzwjGx0p5aew_Xx5TyhBwVzoOdmg_jQOd0aoC-9rS...
4,ma7r5IwEMaxSvvbf0YTfAGmHUkIRNYCZosR-6P93YIuXwQ...
...,...
295,TyD7jQutBqugruEeC0z0_9eusmDXdPOxMdaKJPGJDYJ37o...
296,Mt6pKtktcFEg9Oa8oN2rvmgZgY7LTEBPmcW19KTvWbG90S...
297,M8GoPKXagCB-UOggPfWb6xJFaIhNqJuvH3RbJPp8QBbbXm...
298,DYwPOmdM_BWLeTRtYicQSiiVHwhHmGM_PoSW--6ErBq3Hs...


In [80]:
# link_riotid = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key=RGAPI-{api_key}"
# for i, summoner_id in enumerate(summoners_id):
#     link_puuid = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key=RGAPI-{api_key}"
#     response_puuid = requests.get(link_puuid, headers = headers)
#     try:
#         puuids.append(response_puuid.json()['puuid'])
#     except:
#         print(response_puuid.json())
#         time.sleep(5)

In [10]:
len(puuids)

300

In [2]:
team_comb_data = []
team_members = []
winner = []

In [5]:
puuids_df = pd.read_csv("challengers_puuids.csv")

In [6]:
puuids = list(puuids_df['puuids'])

In [7]:
# puuids = puuids[:2]

In [7]:
# get match data from puuids of users
i = 0
while i < len(puuids):
    puuid = puuids[i]
    link_matchid = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100&api_key=RGAPI-{api_key}"
    response_matchids = requests.get(link_matchid, headers = headers)
    
    
    if response_matchids.status_code != 200:
        print(response_matchids.json())
        time.sleep(120)
        continue

    i += 1
    matches = response_matchids.json()
    j = 0
    while j < len(matches):
            
        matchid = matches[j]
        link_match = f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchid}?api_key=RGAPI-{api_key}"
        response_match = requests.get(link_match, headers = headers)
        if response_match.status_code != 200:
            print(response_match.json())
            time.sleep(120)
            continue
        match_data = response_match.json()
        champ_comb = [md['championId'] for md in match_data['info']['participants']]
        # user_ids = [md["riotIdGameName"] + "#" + md["riotIdTagline"] for md in match_data['info']['participants']]
        team_comb_data.append(champ_comb)
        
        try:
            user_ids = [md["riotIdGameName"] + "#" + md["riotIdTagline"] for md in match_data['info']['participants']]
            team_members.append(user_ids)
            if match_data['info']['teams'][0]['win'] == True:
                winner.append(1)
            else:
                winner.append(0)
        except:
            print(match_data['info']['teams'])
            print('player: ', i)
            print('match: ', j)
            if len(team_members) > len(winner):
                team_members.pop(-1)
            team_comb_data.pop(-1)
            
        
        j += 1

{'status': {'message': 'Rate limit exceeded', 'status_code': 429}}
{'status': {'message': 'Rate limit exceeded', 'status_code': 429}}
{'status': {'message': 'Rate limit exceeded', 'status_code': 429}}


In [35]:
len(team_members)

24840

In [36]:
len(winner)

24836

In [ ]:
len()

In [37]:
i

250

In [38]:
df = pd.DataFrame({'teams' : team_comb_data, "user_ids" : team_members, 'winner' : winner})
# df.to_csv("match_data.csv", index=False)

ValueError: All arrays must be of the same length

In [20]:
df['teams'] = df['teams'].apply(lambda x: ','.join(x))
df['user_ids'] = df['user_ids'].apply(lambda x: ','.join(x))

TypeError: sequence item 0: expected str instance, int found

In [21]:
df

,teams,winner
0,"[114, 203, 61, 50, 201, 72, 163, 777, 119, 40]",0
1,"[72, 141, 157, 268, 50, 58, 64, 39, 222, 117]",1
2,"[266, 154, 105, 429, 432, 897, 421, 136, 221, ...",1
3,"[154, 35, 61, 51, 16, 68, 421, 119, 910, 29]",1
4,"[897, 421, 101, 96, 117, 4, 62, 61, 81, 43]",1
...,...,...
2995,"[4, 64, 268, 157, 111, 68, 120, 136, 221, 497]",1
2996,"[72, 107, 38, 115, 201, 64, 141, 136, 157, 497]",0
2997,"[79, 59, 157, 81, 201, 85, 24, 103, 145, 111]",1
2998,"[126, 234, 1, 145, 526, 62, 421, 29, 96, 201]",1


In [22]:
df.to_csv("match_data.csv", index=False)

In [ ]:
temp = response_match.json()

In [121]:
temp.keys()

dict_keys(['status'])

In [116]:
temp['info']['teams'][0]['win']

False

In [26]:
temp['info']['participants'][0]['championId']

86

In [ ]:
[temp['info']['participants'][_]['championName'] for _ in range(len(temp['info']['participants']))]